In [1]:



import pandas as pd
import numpy as np
import jsonlines
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch_optimizer as optim


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from importlib import reload
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', '{:0.3f}'.format)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.width = 0
import warnings
warnings.filterwarnings('ignore')
import facebook_hateful_memes_detector
reload(facebook_hateful_memes_detector)

from facebook_hateful_memes_detector.utils import read_json_lines_into_df, in_notebook, set_device, get_device
from facebook_hateful_memes_detector.models import Fasttext1DCNNModel, ImageFullTextConvMidFusionModel
from facebook_hateful_memes_detector.preprocessing import TextImageDataset, my_collate, get_datasets, get_image2torchvision_transforms, TextAugment
from facebook_hateful_memes_detector.training import *



In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_device(device)
device


device(type='cuda')

In [3]:
preprocess = get_image2torchvision_transforms()
aug_speeds = {"keyboard": 117, "char_substitute": 109, "char_insert": 109, "char_swap": 114,
              "ocr": 114, "char_delete": 108,
              "word_insert": 0.0, "word_substitute": 0.0, "text_rotate": 32,
              "stopword_insert": 34, "word_join": 32, "word_cutout": 36,
              "w2v_insert": 0.0, "w2v_substitute": 0.0, 
              "fasttext": 137, "glove_twitter": 88, "glove_wiki": 82, "word2vec": 137,
              "synonym": 522, "split": 110, "sentence_shuffle": 67, "one_third_cut": 0.0, "half_cut":0.0}

choice_probas = {"keyboard": 0.1, "char_substitute": 0.0, "char_insert": 0.2, "char_swap": 0.1, "ocr": 0.0, "char_delete": 0.1,
                 "fasttext": 0.0, "glove_twitter": 0.0, "glove_wiki": 0.0, "word2vec": 0.0, "split": 0.2,
                 "stopword_insert": 0.2, "word_join": 0.1, "word_cutout": 0.8,
                 "text_rotate": 0.6, "sentence_shuffle": 0.6, "one_third_cut": 0.25, "half_cut":0.25,
                 "synonym": 0.0,}
preprocess_text = TextAugment([0.55, 0.15, 0.1, 0.2,], choice_probas, fasttext_file="wiki-news-300d-1M-subword.bin")


In [4]:
data = get_datasets(data_dir="../data/", train_text_transform=None, train_image_transform=None, 
                 test_text_transform=None, test_image_transform=None, 
                 cache_images = True, use_images = False, dev=False)


https://discuss.pytorch.org/t/how-to-implement-torch-optim-lr-scheduler-cosineannealinglr/28797/11

# Params

In [5]:
sgd = torch.optim.SGD
sgd_params = dict(lr=2e-2, momentum=0.9, dampening=0, weight_decay=0, nesterov=False)

rangerQH = optim.RangerQH
rangerQHparams = dict(lr=1e-3, betas=(0.9, 0.999), nus=(.7, 1.0),
    weight_decay=0.0,
    k=6,
    alpha=.5,
    decouple_weight_decay=True,
    eps=1e-8,)

adam = torch.optim.Adam
adam_params = params=dict(lr=1e-3, weight_decay=1e-6)

adamw = torch.optim.AdamW
adamw_params = dict(lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-2)

novograd = optim.NovoGrad
novograd_params = dict(lr= 1e-3,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,
    grad_averaging=False,
    amsgrad=False,)

qhadam = optim.QHAdam
qhadam_params = dict(lr= 1e-3,
    betas=(0.9, 0.999),
    nus=(1.0, 1.0),
    weight_decay=0,
    decouple_weight_decay=False,
    eps=1e-8,)

radam = optim.RAdam
radam_params = dict(lr= 1e-3,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,)

yogi = optim.Yogi
yogi_params = dict(lr= 1e-2,
    betas=(0.9, 0.999),
    eps=1e-3,
    initial_accumulator=1e-6,
    weight_decay=0)




In [9]:
batch_size=512
epochs = 10
optimizer = adam
optimizer_params = adam_params

scheduler_init_fn = get_multistep_lr([11, 13], gamma=0.25) # get_cosine_schedule_with_warmup # get_cosine_with_hard_restarts_schedule_with_warmup
scheduler_init_fn = get_cosine_schedule_with_warmup()
augmentation_weights = {"None": 1.0}



# Fasttext 1D CNN

## 512 DIMS

In [12]:
model_fn = model_builder(Fasttext1DCNNModel, 
                         dict(classifier_dims=256, num_classes=2, 
                              n_layers=2, final_layer_builder=fb_1d_loss_builder,
                              gaussian_noise=0.2, dropout=0.1, embedding_dims=256, internal_dims=512,
                             fasttext_file="crawl-300d-2M-subword.bin", featurizer="transformer",),
                         optimiser_class=optimizer, optimiser_params=optimizer_params)





n_tests = 1
multi_eval=False 
kfold=False
results, prfs = train_validate_ntimes(model_fn, data, batch_size, epochs, 
                                      augmentation_weights=augmentation_weights, 
                                      kfold=kfold, multi_eval=multi_eval,
                                      scheduler_init_fn=scheduler_init_fn, 
                                      validation_epochs=[1, 4, 7, 9, 11, 14, 17, 19])
r1, p1 = results, prfs
results
prfs
 
# Defualt
# 0.749 	0.717, gaussian_noise=0.2, dropout=0.0, weight_decay=1e-5
# 0.735 	0.703, gaussian_noise=0.3, dropout=0.0, weight_decay=1e-5
# 0.750 	0.715, gaussian_noise=0.3, dropout=0.0, weight_decay=1e-7
# 0.722 	0.701, gaussian_noise=0.2, dropout=0.0, weight_decay=1e-4
# 0.722 	0.701, gaussian_noise=0.05, dropout=0.0, weight_decay=1e-4
# 0.725 	0.704, gaussian_noise=0.0, dropout=0.0, weight_decay=1e-4
# 0.731 	0.699, gaussian_noise=0.0, dropout=0.0, weight_decay=5e-5

# 5 size cnn
# 0.743 	0.707, gaussian_noise=0.2, dropout=0.0, weight_decay=1e-5
# 0.742 	0.709, gaussian_noise=0.3, dropout=0.0, weight_decay=1e-5
# 0.739 	0.710, gaussian_noise=0.5, dropout=0.0, weight_decay=1e-5
# 0.733 	0.705, gaussian_noise=0.8, dropout=0.0, weight_decay=1e-5

# Simple CNN
# 0.616 	0.607, gaussian_noise=0.8, dropout=0.0, weight_decay=1e-5
# 0.694 	0.685, gaussian_noise=0.0, dropout=0.0, weight_decay=1e-7

# Relu only  
# 0.744 	0.712, gaussian_noise=0.2, dropout=0.0, weight_decay=1e-5
# 0.740 	0.704, gaussian_noise=0.3, dropout=0.0, weight_decay=1e-5
# 0.744 	0.710, gaussian_noise=0.4, dropout=0.0, weight_decay=1e-5
# 0.741 	0.711, gaussian_noise=0.5, dropout=0.0, weight_decay=1e-5
# 0.734 	0.704, gaussian_noise=0.4, dropout=0.0, weight_decay=1e-9

# Gelu
# 0.738 	0.711, gaussian_noise=0.5, dropout=0.0, weight_decay=1e-6
# 0.731 	0.707,gaussian_noise=0.3, dropout=0.0, weight_decay=1e-5 

# GRU 
# 0.728 	0.708, gaussian_noise=0.3, dropout=0.0, weight_decay=1e-5
# 0.724 	0.704, gaussian_noise=0.0, dropout=0.0, weight_decay=1e-5

# 0.777 	0.729 No layer norm or unit norm
# 0.761 	0.730, 0.013, with layer norm
# 0.769 	0.725

# 0.800 	0.715 8m 10s
# 0.771 	0.725 7m 25s

# 0.753 	0.716 7m 60s 7826736

# 0.729 	0.707 7m 16s 6417712

# 0.793 	0.717 6m 31s 5532976

# 0.750 	0.713 6m 21s 5532976


# GRU
# 17m 14s 0.906 	0.650 10316576

# 15m 24s 0.907 	0.635 8743712

# 11m 41s 0.925 	0.627 2904864


Model Params = 15065088 
 Fasttext1DCNNModel(
  (crawl_nn): ExpandContract(
    (nn): Sequential(
      (0): Transpose()
      (1): Dropout(p=0.1, inplace=False)
      (2): Conv1d(600, 512, kernel_size=(1,), stride=(1,), groups=8, bias=False)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Dropout(p=0.1, inplace=False)
      (5): Conv1d(512, 256, kernel_size=(1,), stride=(1,), groups=4, bias=False)
      (6): Transpose()
      (7): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
  )
  (featurizer): TransformerFeaturizer(
    (input_nn): Linear(in_features=256, out_features=512, bias=False)
    (output_nn): Linear(in_features=512, out_features=256, bias=False)
    (transformer): Transformer(
      (encoder): TransformerEncoder(
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (linear1): Linear(in_featu


Epoch =  1 Loss = 1.393860 LR = 0.00033333
Epoch =  1 Train = 0.627022 Val = 0.609762



Epoch =  2 Loss = 0.917501 LR = 0.00066667



Epoch =  3 Loss = 0.860071 LR = 0.00100000



Epoch =  4 Loss = 0.763659 LR = 0.00095048
Epoch =  4 Train = 0.689001 Val = 0.684763


...


Epoch =  8 Loss = 0.649199 LR = 0.00018826



Epoch =  9 Loss = 0.626209 LR = 0.00004952
Epoch =  9 Train = 0.759587 Val = 0.725161



Epoch =  10 Loss = 0.639540 LR = 0.00000000



train   val
mean_or_std metric               
mean        map       0.640 0.600
            accuracy  0.681 0.657
            auc       0.760 0.725
std         map       0.000 0.000
            accuracy  0.000 0.000
            auc       0.000 0.000

train                             val                      
    precision recall    f1  supoort precision recall    f1  supoort
neg     0.807  0.653 0.722 4560.000     0.783  0.635 0.701 1140.000
pos     0.549  0.730 0.627 2640.000     0.525  0.695 0.598  660.000

# Predict

In [ ]:
model_fn = model_builder(Fasttext1DCNNModel, 
                         dict(classifer_dims=256, num_classes=2, 
                              gaussian_noise=0.3, dropout=0.2, embedding_dims=512, internal_dims=256,
                             fasttext_file="crawl-300d-2M-subword.bin", classifier="transformer",),
                         optimiser_class=optimizer, optimiser_params=optimizer_params)


In [ ]:
batch_size=512
epochs = 1

submission, text_model = train_and_predict(model_fn, data, batch_size, epochs, augmentation_weights, scheduler_init_fn=scheduler_init_fn)
submission.to_csv("submission.csv",index=False)
submission.sample(3)


In [ ]:
submission.sample(10)